In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#file = "Exam Sched Prog Datasets.xlsx"
file = r"C:\Users\William Hazen\Documents\GitHub\exam-timetabling-smac\Exam Sched Prog Datasets.xlsx"
rooms = pd.read_excel(file, sheet_name = "datasets room caps ")
courses = pd.read_excel(file, sheet_name = "20221 course size")
enrolments = pd.read_excel(file, sheet_name = "20221 anonymized enrolments")
schedule = pd.read_excel(file, sheet_name = "20221 Final Schedule_fromLSM")

In [3]:
OG_Course_list = schedule.Course.dropna()
OG_Course_list = OG_Course_list.values

# Data Extraction

#### Random sample of ID's

In [4]:
qwe = enrolments["HASHED_PERSON_ID"].values
qwer = np.unique(qwe)
#random_sample_names_generator = (qwer[i] for i in range(np.random.randint(0, len(qwer)-1)))
random_sample_names_generator = (qwer[i] for i in range(500))
name = []

In [5]:
for i in random_sample_names_generator:
    name.append(i)
    
Student_ID_List = random.sample(name, len(name))
print(len(Student_ID_List))

500


#### Unique Exams for given student

In [6]:
Student_ID_Exams = []
#enrolments["HASHED_PERSON_ID"].loc[ID]
for ID in Student_ID_List:
    Student_ID_Exams.append(enrolments["ACAD_ACT_CD"][enrolments.eq(ID).any(1)].values)

C:\Users\William Hazen\AppData\Local\Temp\ipykernel_17980\1881659361.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  Student_ID_Exams.append(enrolments["ACAD_ACT_CD"][enrolments.eq(ID).any(1)].values)


In [7]:
Student_ID_Exams
exam_list = []
for exam in Student_ID_Exams:
    exam_list.append(exam.any())
    
uniq_exams = np.unique(exam_list)

In [8]:
new_enrol = pd.DataFrame(enrolments).copy()
new_enrol["class"] = 1

temp_df = new_enrol[new_enrol["HASHED_PERSON_ID"].isin(Student_ID_List)]
final_temp = temp_df[temp_df["ACAD_ACT_CD"].isin(uniq_exams)]

#### Course Enrollment

In [9]:
courses_enrollments_set = pd.pivot_table(data=final_temp, values="class", index="ACAD_ACT_CD", columns=["HASHED_PERSON_ID"], fill_value=0)

# Cleaning Room Table

In [10]:
#Cleaning room table

rooms['Room'] = rooms['Room'].astype(str)
rooms['Room'] = rooms['Bd'].str.cat(rooms['Room'], sep = " ")

rooms = rooms.drop(['Note','Bd'], axis=1)

# Cleaning Course Data

In [11]:
#Cleaning course data

courses['COURSE_CODE'] = courses['COURSE_CODE'].str.cat(courses['SECTIONCD'], sep = "")
courses = courses.drop(['SESSION_CD','SECTIONCD','ADMINFACULTYCODE','ADMINDEPT'], axis=1)
crselist = courses['COURSE_CODE'].tolist()
sizelist = courses['CURR_REG_QTY'].tolist()

# Enrolment data

In [12]:
idlist = enrolments['HASHED_PERSON_ID'].tolist()
classlist = enrolments['ACAD_ACT_CD'].tolist()

schedule['Room'] = schedule['Room'].astype(str)
schedule['Examdate'] = schedule['Examdate'].astype(str)
schedule['Begin'] = schedule['Begin'].astype(str)
schedule['Ends'] = schedule['Ends'].astype(str)

schedule['Room'] = schedule['Bd'].str.cat(schedule['Room'], sep = " ")

schedule = schedule.drop(['Bd','Faculty','Department','Section','Sesson','Course Name','Duration'], axis=1)

schedule.replace('ZZ REFER_TO_FACULTY_SCHEDULE', 'None', inplace=True)
schedule.replace('ZZ REFER_TO_FACULTY_SCHEDULE', 'None', inplace=True)
schedule.replace('ZZ ONLIN', 'None', inplace=True)
schedule.replace('APSCDept APSCDept', 'None', inplace=True)
schedule.replace('APSCDept ComputerLab', 'None', inplace=True)
schedule.replace('ZZ KNOX', 'None', inplace=True)
schedule.replace('ZZ VLAD', 'None', inplace=True)
schedule.replace('ZZ ONLINMUSIC', 'None', inplace=True)


In [13]:
room_list = schedule['bd_room'].dropna().values
test_list = room_list.tolist()
roomlist = [i for i in test_list if i != "None"]

In [14]:
room_cap_list = schedule['Room Cap'].dropna().values
test_cap_list = room_cap_list.tolist()
roomcaplist = [i for i in test_cap_list if i != "None"]

In [15]:
print(len(roomcaplist), len(roomlist))

1279 1279


In [16]:
OG_CL = list(OG_Course_list)

unique_course_list=[]
for course in OG_CL:
    for crs in uniq_exams:
        if course[0:8] == crs:
            unique_course_list.append(course) 
    

In [17]:
new_schedule = pd.DataFrame(schedule).copy()
temp_sch_df = new_schedule[new_schedule["Course"].isin(unique_course_list)]

In [31]:
examdate_time_list = (schedule['Examdate'].dropna().astype(str) + " : " + schedule['Begin']).dropna().tolist()
examdate_time_list = np.unique(examdate_time_list)
examdate_time_list = [i for i in examdate_time_list if i != "NaT : nan"]

In [32]:
examdate_time_list

['2022-04-11 : 09:00:00',
 '2022-04-11 : 14:00:00',
 '2022-04-11 : 19:00:00',
 '2022-04-12 : 09:00:00',
 '2022-04-12 : 14:00:00',
 '2022-04-12 : 19:00:00',
 '2022-04-13 : 09:00:00',
 '2022-04-13 : 14:00:00',
 '2022-04-13 : 19:00:00',
 '2022-04-14 : 09:00:00',
 '2022-04-14 : 14:00:00',
 '2022-04-14 : 19:00:00',
 '2022-04-18 : 09:00:00',
 '2022-04-18 : 14:00:00',
 '2022-04-18 : 19:00:00',
 '2022-04-19 : 09:00:00',
 '2022-04-19 : 09:30:00',
 '2022-04-19 : 14:00:00',
 '2022-04-19 : 18:30:00',
 '2022-04-19 : 19:00:00',
 '2022-04-20 : 09:00:00',
 '2022-04-20 : 09:30:00',
 '2022-04-20 : 14:00:00',
 '2022-04-20 : 18:30:00',
 '2022-04-20 : 19:00:00',
 '2022-04-21 : 09:00:00',
 '2022-04-21 : 09:30:00',
 '2022-04-21 : 14:00:00',
 '2022-04-21 : 18:30:00',
 '2022-04-21 : 19:00:00',
 '2022-04-22 : 09:00:00',
 '2022-04-22 : 09:30:00',
 '2022-04-22 : 14:00:00',
 '2022-04-22 : 18:30:00',
 '2022-04-22 : 19:00:00',
 '2022-04-23 : 09:00:00',
 '2022-04-23 : 09:30:00',
 '2022-04-23 : 14:00:00',
 '2022-04-25

In [33]:
#Schedule data in list format

schdcrselist = temp_sch_df['Course'].tolist()
schdroomlist = temp_sch_df['Room'].tolist()
exmdatelist = temp_sch_df['Examdate'].tolist()
starttimelist = temp_sch_df['Begin'].tolist()
endtimelist = temp_sch_df['Ends'].tolist()
enrllist = temp_sch_df['Enrolment'].tolist()
crseinroomlist = temp_sch_df['Course In Room'].tolist()
totalocclist = temp_sch_df['Total Occupancy'].tolist()
examdate_time_list = (temp_sch_df['Examdate'].astype(str) + " : " + temp_sch_df['Begin']).tolist()
course_enrollment_values = courses_enrollments_set.drop(["TEP445H1"], axis=0).values
sumHe_s = np.sum(course_enrollment_values, axis=1)
temp_roomcaplist = sumHe_s + 4

In [34]:
print("Exam set:" '\n', schdcrselist,'\n\n', 
      "Course Enrollement Set:" '\n',course_enrollment_values, '\n\n', 
      "Number of students in course:" '\n',sumHe_s, '\n\n', 
      "Room Capacity set:" '\n',roomcaplist, '\n\n'
)

Exam set:
 ['AER302H1S', 'AER372H1S', 'APS105H1S', 'APS105H1S', 'APS106H1S', 'APS106H1S', 'APS106H1S', 'APS112H1S', 'APS113Y1Y', 'APS305H1S', 'APS360H1S', 'APS420H1S', 'APS502H1S', 'BME205H1S', 'BME205H1S', 'BME330H1S', 'BME331H1S', 'BME346H1S', 'CHE354H1S', 'CHE507H1S', 'CIV185H1S', 'CIV209H1S', 'CIV214H1S', 'CIV300H1S', 'CIV313H1S', 'CIV324H1S', 'CIV440H1S', 'ECE110H1S', 'ECE110H1S', 'ECE110H1S', 'ECE159H1S', 'ECE216H1S', 'ECE216H1S', 'ECE221H1S', 'ECE221H1S', 'ECE221H1S', 'ECE231H1S', 'ECE231H1S', 'ECE259H1S', 'ECE302H1S', 'ECE311H1S', 'ECE313H1S', 'ECE318H1S', 'ECE330H1S', 'ECE334H1S', 'ECE334H1S', 'ECE342H1S', 'ECE353H1S', 'ECE356H1S', 'ECE419H1S', 'ECE419H1S', 'ECE421H1S', 'ECE437H1S', 'ECE472H1S', 'ECE472H1S', 'ECE568H1S', 'FOR424H1S', 'FOR424H1S', 'HPS283H1S', 'JRE300H1S', 'JRE410H1S', 'JRE420H1S', 'MAT234H1S', 'MIE210H1S', 'MIE210H1S', 'MIE210H1S', 'MIE210H1S', 'MIE237H1S', 'MIE237H1S', 'MIE253H1S', 'MIE253H1S', 'MIE304H1S', 'MIE311H1S', 'MIE313H1S', 'MIE315H1S', 'MIE315H1S', 

In [35]:
print("Exam Date and time slot:" '\n',examdate_time_list,'\n\n',
      "Room Set:" '\n',roomlist, '\n\n', 
      "Student set:" '\n', Student_ID_List,'\n\n')

Exam Date and time slot:
 ['2022-04-22 : 14:00:00', '2022-04-27 : 18:30:00', '2022-04-23 : 09:30:00', '2022-04-23 : 09:30:00', '2022-04-23 : 09:30:00', '2022-04-23 : 09:30:00', '2022-04-23 : 09:30:00', '2022-04-19 : 09:30:00', '2022-04-19 : 09:30:00', '2022-04-28 : 14:00:00', '2022-04-21 : 14:00:00', '2022-04-19 : 18:30:00', '2022-04-29 : 09:30:00', '2022-04-26 : 09:30:00', '2022-04-26 : 09:30:00', '2022-04-25 : 18:30:00', '2022-04-26 : 14:00:00', '2022-04-28 : 18:30:00', '2022-04-29 : 14:00:00', '2022-04-23 : 14:00:00', '2022-04-21 : 09:30:00', '2022-04-28 : 09:30:00', '2022-04-19 : 09:30:00', '2022-04-29 : 14:00:00', '2022-04-27 : 14:00:00', '2022-04-23 : 14:00:00', '2022-04-25 : 14:00:00', '2022-04-21 : 09:30:00', '2022-04-21 : 09:30:00', '2022-04-21 : 09:30:00', '2022-04-21 : 14:00:00', '2022-04-28 : 14:00:00', '2022-04-28 : 14:00:00', '2022-04-20 : 09:30:00', '2022-04-20 : 09:30:00', '2022-04-20 : 09:30:00', '2022-04-25 : 09:30:00', '2022-04-25 : 09:30:00', '2022-04-22 : 18:30:00'

In [36]:
import pickle

In [37]:
# print("Exam set:" '\n', schdcrselist,'\n\n', 
#       "Course Enrollement Set:" '\n',course_enrollment_values, '\n\n', 
#       "Number of students in course:" '\n',sumHe_s, '\n\n', 
#       "Room Capacity set:" '\n',roomcaplist, '\n\n'
# )
# print("Exam Date and time slot:" '\n',examdate_time_list,'\n\n',
#       "Room Set:" '\n',crseinroomlist, '\n\n', 
#       "Student set:" '\n', Student_ID_List,'\n\n')

In [38]:
with open('Exam_set_Pickle.pkl', 'wb') as f:
    pickle.dump(schdcrselist, f)

with open('Student_set_Pickle.pkl', 'wb') as f:
    pickle.dump(Student_ID_List, f)
    
with open('Timeslots_Pickle.pkl', 'wb') as f:
    pickle.dump(examdate_time_list, f)
    
with open('Rooms_Pickle.pkl', 'wb') as f:
    pickle.dump(RL, f)

with open('course enrolments_Pickle.pkl', 'wb') as f:
    pickle.dump(course_enrollment_values, f)
    
with open('Room_Capacity.pkl', 'wb') as f:
    pickle.dump(roomcaplist, f)


NameError: name 'RL' is not defined